## BUSINESS PROBLEM

UK-based retail company wants to segment its customers and determine a roadmap for sales and marketing activities. It thinks that marketing activities specific to customer segments that exhibit common behaviors will increase revenue.
RFM analysis will be used for segmentation. In addition, in order for the company to make a medium and long term plan, the existing customers
estimate the potential value they will provide to the company in the future.

### DATASET BACKGROUND

The dataset Online Retail II contains online sales transactions of a UK-based retail company between 01/12/2009 and 09/12/2011. The company's product catalog includes souvenirs and most of its customers are wholesalers.

8 Variable, 541.909 Observation

* InvoiceNo : Invoice Number (If this code starts with C, it means that the transaction was canceled)
* StockCode : Product Code (Unique for each product)
* Description : Product Name
* Quantity : Number of Products (How many of the products in the invoices were sold)
* InvoiceDate : Invoice Date
* UnitPrice : Invoice Price (£)
* CustomerID : Unique Customer Number
* Country : Country Name

### IMPORTINGS

In [1]:
!pip install lifetimes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df_ = pd.read_excel("/kaggle/input/online-retail-ii-dataset/online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()

### FUNCTIONS

In [3]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

### DATA UNDERSTANDING & PREPARATION

In [4]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [6]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [7]:
df.dropna(inplace=True)

In [8]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [9]:
df["StockCode"].nunique()

3684

In [10]:
df.groupby("Description").agg({"Quantity" : "sum"})

,Quantity
Description,
4 PURPLE FLOCK DINNER CANDLES,140
50'S CHRISTMAS GIFT BAG LARGE,1883
DOLLY GIRL BEAKER,2391
I LOVE LONDON MINI BACKPACK,360
I LOVE LONDON MINI RUCKSACK,1
...,...
ZINC T-LIGHT HOLDER STARS SMALL,4850
ZINC TOP 2 DOOR WOODEN SHELF,5
ZINC WILLIE WINKIE CANDLE STICK,2595


In [11]:
df.groupby("Description").agg({"Invoice" : "count"}).sort_values("Invoice", ascending=False).head(5)

,Invoice
Description,
WHITE HANGING HEART T-LIGHT HOLDER,2070
REGENCY CAKESTAND 3 TIER,1905
JUMBO BAG RED RETROSPOT,1662
ASSORTED COLOUR BIRD ORNAMENT,1418
PARTY BUNTING,1416


In [12]:
df = df[~df["Invoice"].str.contains("C", na=False)]

In [13]:
df["TotalPrice"] = df["Quantity"] * df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom,15.300
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom,22.000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340


### CALCULATING RFM METRICS

In [14]:
today_date = df["InvoiceDate"].max() + dt.timedelta(days=2)

In [15]:
rfm = df.groupby("Customer ID").agg({"InvoiceDate" : lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     "Invoice" : lambda Invoice: Invoice.nunique(),
                                     "TotalPrice" : lambda TotalPrice: TotalPrice.sum()})

rfm.columns = ["recency", "frequency", "monetary"]
rfm.reset_index(inplace=True)
rfm.head()

,Customer ID,recency,frequency,monetary
0,12346.000,327,1,77183.600
1,12347.000,3,7,4310.000
2,12348.000,76,4,1797.240
3,12349.000,20,1,1757.550
4,12350.000,311,1,334.400


### CALCULATING RFM SCORES

In [16]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, [5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, [1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, [1, 2, 3, 4, 5])

rfm["RF_Score"] = rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str)

rfm.head()

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_Score
0,12346.000,327,1,77183.600,1,1,5,11
1,12347.000,3,7,4310.000,5,5,5,55
2,12348.000,76,4,1797.240,2,4,4,24
3,12349.000,20,1,1757.550,4,1,4,41
4,12350.000,311,1,334.400,1,1,2,11


### CREATING & ANALYSING RFM SEGMENTS

In [17]:
seg_map = {r'[1-2][1-2]': 'hibernating',
           r'[1-2][3-4]': 'at_Risk',
           r'[1-2]5': 'cant_loose',
           r'3[1-2]': 'about_to_sleep',
           r'33': 'need_attention',
           r'[3-4][4-5]': 'loyal_customers',
           r'41': 'promising',
           r'51': 'new_customers',
           r'[4-5][2-3]': 'potential_loyalists',
           r'5[4-5]': 'champions'}

rfm["Segment"] = rfm["RF_Score"].replace(seg_map, regex=True)

rfm.head()

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_Score,Segment
0,12346.000,327,1,77183.600,1,1,5,11,hibernating
1,12347.000,3,7,4310.000,5,5,5,55,champions
2,12348.000,76,4,1797.240,2,4,4,24,at_Risk
3,12349.000,20,1,1757.550,4,1,4,41,promising
4,12350.000,311,1,334.400,1,1,2,11,hibernating


### SELECTING IMPORTANT SEGMENTS & ANALYSIS

In [18]:
rfm_selected = rfm[rfm["Segment"].isin(["hibernating", "champions", "at_Risk"])]

rfm_selected.groupby("Segment").agg({"recency" : ["max", "min", "count", "mean"],
                                     "frequency" : ["max", "min", "count", "mean"],
                                     "monetary" : ["max", "min", "count", "mean"]})



recency                   frequency                    monetary  \
                max min count    mean       max min count   mean        max   
Segment                                                                       
at_Risk         374  74   580 156.062         6   2   580  2.866  44534.300   
champions        14   2   633   6.877       210   3   633 12.417 280206.020   
hibernating     375  74  1065 218.898         2   1  1065  1.101  77183.600   

                                    
                min count     mean  
Segment                             
at_Risk      52.000   580 1076.506  
champions   201.120   633 6857.964  
hibernating   3.750  1065  487.708

In [19]:
rfm_loyal = rfm[rfm["Segment"].isin(["loyal_customers"])]["Customer ID"].astype("int").reset_index(drop=True)
rfm_loyal.head()

0    12352
1    12359
2    12370
3    12380
4    12388
Name: Customer ID, dtype: int64

### CLTV PREDICTION

In [20]:
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

In [21]:
today_date = df["InvoiceDate"].max() + dt.timedelta(days=2)

### PREPARATION of DATA STRUCTURE

In [22]:
cltv_df = pd.DataFrame()

cltv_df["recency_cltv_weekly"] = df.groupby("Customer ID").agg({"InvoiceDate" : lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days}) / 7
cltv_df["T_weekly"] = df.groupby("Customer ID").agg({"InvoiceDate" : lambda InvoiceDate: (today_date - InvoiceDate.min()).days}) / 7
cltv_df["frequency"] = df.groupby("Customer ID").agg({"Invoice" : lambda Invoice: Invoice.nunique()})
cltv_df["monetary"] = df.groupby("Customer ID").agg({"TotalPrice": lambda TotalPrice: TotalPrice.sum()})
cltv_df["monetary_cltv_avg"] = cltv_df["monetary"] / cltv_df["frequency"]
cltv_df.reset_index()

cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

cltv_df.head()

,recency_cltv_weekly,T_weekly,frequency,monetary,monetary_cltv_avg
Customer ID,,,,,
12347.000,52.143,52.571,7,4310.000,615.714
12348.000,40.286,51.286,4,1797.240,449.310
12352.000,37.143,42.571,8,2506.040,313.255
12356.000,43.143,46.714,3,2811.430,937.143
12358.000,21.286,21.714,2,1168.060,584.030


In [23]:
cltv_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2845 entries, 12347.0 to 18287.0
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   recency_cltv_weekly  2845 non-null   float64
 1   T_weekly             2845 non-null   float64
 2   frequency            2845 non-null   int64  
 3   monetary             2845 non-null   float64
 4   monetary_cltv_avg    2845 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 133.4 KB


### FITTING BG-NBD MODEL

In [24]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df["frequency"],
        cltv_df["recency_cltv_weekly"],
        cltv_df["T_weekly"])

<lifetimes.BetaGeoFitter: fitted with 2845 subjects, a: 0.12, alpha: 11.44, b: 2.50, r: 2.18>

In [25]:
cltv_df["exp_sales_6_month"] = bgf.predict(4 * 6,
                                           frequency = cltv_df["frequency"],
                                           recency = cltv_df["recency_cltv_weekly"],
                                           T = cltv_df["T_weekly"])

cltv_df.head()

,recency_cltv_weekly,T_weekly,frequency,monetary,monetary_cltv_avg,exp_sales_6_month
Customer ID,,,,,,
12347.000,52.143,52.571,7,4310.000,615.714,3.322
12348.000,40.286,51.286,4,1797.240,449.310,2.160
12352.000,37.143,42.571,8,2506.040,313.255,4.258
12356.000,43.143,46.714,3,2811.430,937.143,2.015
12358.000,21.286,21.714,2,1168.060,584.030,2.811


### FITTING GAMMA-GAMMA MODEL

In [26]:
ggf = GammaGammaFitter(penalizer_coef=0.005)
ggf.fit(cltv_df["frequency"],
        cltv_df["monetary_cltv_avg"])

<lifetimes.GammaGammaFitter: fitted with 2845 subjects, p: 5.62, q: 0.44, v: 5.54>

In [27]:
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(frequency = cltv_df["frequency"],
                                                                       monetary_value = cltv_df["monetary_cltv_avg"])

cltv_df.head()

,recency_cltv_weekly,T_weekly,frequency,monetary,monetary_cltv_avg,exp_sales_6_month,exp_average_value
Customer ID,,,,,,,
12347.000,52.143,52.571,7,4310.000,615.714,3.322,625.367
12348.000,40.286,51.286,4,1797.240,449.310,2.160,462.156
12352.000,37.143,42.571,8,2506.040,313.255,4.258,317.889
12356.000,43.143,46.714,3,2811.430,937.143,2.015,971.098
12358.000,21.286,21.714,2,1168.060,584.030,2.811,617.422


### CALCULATION of CLTV with BG-NBD and GG MODEL

In [28]:
cltv_df["6m_cltv"] = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency_cltv_weekly"],
                                   cltv_df["T_weekly"],
                                   cltv_df["monetary_cltv_avg"],
                                   time=6,
                                   freq="W",
                                   discount_rate=0.01)

cltv_df = cltv_df.sort_values("6m_cltv", ascending=False).reset_index()
cltv_df.head(20)

,Customer ID,recency_cltv_weekly,T_weekly,frequency,monetary,monetary_cltv_avg,exp_sales_6_month,exp_average_value,6m_cltv
0,16446.000,29.143,29.571,2,168472.500,84236.250,2.288,88634.665,212336.646
1,14646.000,50.429,50.857,73,280206.020,3838.439,28.296,3843.737,113960.646
2,18102.000,52.286,52.571,60,259657.300,4327.622,22.791,4334.880,103517.482
3,17450.000,51.286,52.714,46,194550.790,4229.365,17.534,4238.625,77874.797
4,14096.000,13.857,14.714,17,65164.790,3833.223,16.643,3856.047,67162.312
5,12415.000,44.714,48.429,21,124914.530,5948.311,8.885,5976.804,55639.508
6,14911.000,53.143,53.429,201,143825.060,715.548,73.592,715.928,55206.114
7,14156.000,51.571,53.286,55,117379.630,2134.175,20.581,2138.132,46109.395
8,16000.000,0.000,0.571,3,12393.700,4131.233,9.224,4274.401,41201.923
9,17511.000,52.857,53.571,31,91062.380,2937.496,11.944,2947.104,36882.990


### 1M-12M CLTV VALUES COMPARISON

In [29]:
cltv_df["1m_cltv"] = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency_cltv_weekly"],
                                   cltv_df["T_weekly"],
                                   cltv_df["monetary_cltv_avg"],
                                   time=1,
                                   freq="W",
                                   discount_rate=0.01)

cltv_df["12m_cltv"] = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency_cltv_weekly"],
                                   cltv_df["T_weekly"],
                                   cltv_df["monetary_cltv_avg"],
                                   time=12,
                                   freq="W",
                                   discount_rate=0.01)


In [30]:
cltv_df.sort_values("1m_cltv", ascending=False).head(10)

,Customer ID,recency_cltv_weekly,T_weekly,frequency,monetary,monetary_cltv_avg,exp_sales_6_month,exp_average_value,6m_cltv,1m_cltv,12m_cltv
0,16446.000,29.143,29.571,2,168472.500,84236.250,2.288,88634.665,212336.646,37304.052,402225.954
1,14646.000,50.429,50.857,73,280206.020,3838.439,28.296,3843.737,113960.646,19821.753,217496.656
2,18102.000,52.286,52.571,60,259657.300,4327.622,22.791,4334.880,103517.482,17998.215,197628.737
3,17450.000,51.286,52.714,46,194550.790,4229.365,17.534,4238.625,77874.797,13539.885,148672.750
4,14096.000,13.857,14.714,17,65164.790,3833.223,16.643,3856.047,67162.312,11905.526,126505.647
5,12415.000,44.714,48.429,21,124914.530,5948.311,8.885,5976.804,55639.508,9686.284,106113.512
6,14911.000,53.143,53.429,201,143825.060,715.548,73.592,715.928,55206.114,9595.651,105421.283
7,14156.000,51.571,53.286,55,117379.630,2134.175,20.581,2138.132,46109.395,8015.618,88040.247
8,16000.000,0.000,0.571,3,12393.700,4131.233,9.224,4274.401,41201.923,7514.395,76498.587
9,17511.000,52.857,53.571,31,91062.380,2937.496,11.944,2947.104,36882.990,6411.953,70421.314


In [31]:
cltv_df.sort_values("12m_cltv", ascending=False).head(10)

,Customer ID,recency_cltv_weekly,T_weekly,frequency,monetary,monetary_cltv_avg,exp_sales_6_month,exp_average_value,6m_cltv,1m_cltv,12m_cltv
0,16446.000,29.143,29.571,2,168472.500,84236.250,2.288,88634.665,212336.646,37304.052,402225.954
1,14646.000,50.429,50.857,73,280206.020,3838.439,28.296,3843.737,113960.646,19821.753,217496.656
2,18102.000,52.286,52.571,60,259657.300,4327.622,22.791,4334.880,103517.482,17998.215,197628.737
3,17450.000,51.286,52.714,46,194550.790,4229.365,17.534,4238.625,77874.797,13539.885,148672.750
4,14096.000,13.857,14.714,17,65164.790,3833.223,16.643,3856.047,67162.312,11905.526,126505.647
5,12415.000,44.714,48.429,21,124914.530,5948.311,8.885,5976.804,55639.508,9686.284,106113.512
6,14911.000,53.143,53.429,201,143825.060,715.548,73.592,715.928,55206.114,9595.651,105421.283
7,14156.000,51.571,53.286,55,117379.630,2134.175,20.581,2138.132,46109.395,8015.618,88040.247
8,16000.000,0.000,0.571,3,12393.700,4131.233,9.224,4274.401,41201.923,7514.395,76498.587
9,17511.000,52.857,53.571,31,91062.380,2937.496,11.944,2947.104,36882.990,6411.953,70421.314


### COUNTRY BASED CLTV SEGMENTATION

In [32]:
cltv_df = pd.merge(cltv_df, df[["Customer ID", "Country"]], on="Customer ID", how="inner").drop_duplicates()
cltv_df = cltv_df[cltv_df["Country"].isin(["United Kingdom"])]

cltv_df.head()

,Customer ID,recency_cltv_weekly,T_weekly,frequency,monetary,monetary_cltv_avg,exp_sales_6_month,exp_average_value,6m_cltv,1m_cltv,12m_cltv,Country
0,16446.000,29.143,29.571,2,168472.500,84236.250,2.288,88634.665,212336.646,37304.052,402225.954,United Kingdom
2079,18102.000,52.286,52.571,60,259657.300,4327.622,22.791,4334.880,103517.482,17998.215,197628.737,United Kingdom
2510,17450.000,51.286,52.714,46,194550.790,4229.365,17.534,4238.625,77874.797,13539.885,148672.750,United Kingdom
2847,14096.000,13.857,14.714,17,65164.790,3833.223,16.643,3856.047,67162.312,11905.526,126505.647,United Kingdom
15747,16000.000,0.000,0.571,3,12393.700,4131.233,9.224,4274.401,41201.923,7514.395,76498.587,United Kingdom


In [33]:
cltv_df["Segment"] = pd.qcut(cltv_df["6m_cltv"], 4, ["D", "C", "B", "A"])

cltv_df.head()

,Customer ID,recency_cltv_weekly,T_weekly,frequency,monetary,monetary_cltv_avg,exp_sales_6_month,exp_average_value,6m_cltv,1m_cltv,12m_cltv,Country,Segment
0,16446.000,29.143,29.571,2,168472.500,84236.250,2.288,88634.665,212336.646,37304.052,402225.954,United Kingdom,A
2079,18102.000,52.286,52.571,60,259657.300,4327.622,22.791,4334.880,103517.482,17998.215,197628.737,United Kingdom,A
2510,17450.000,51.286,52.714,46,194550.790,4229.365,17.534,4238.625,77874.797,13539.885,148672.750,United Kingdom,A
2847,14096.000,13.857,14.714,17,65164.790,3833.223,16.643,3856.047,67162.312,11905.526,126505.647,United Kingdom,A
15747,16000.000,0.000,0.571,3,12393.700,4131.233,9.224,4274.401,41201.923,7514.395,76498.587,United Kingdom,A
